In [25]:
import sys
from pathlib import Path
# Add parent directory to Python path
root_dir = Path.cwd().parent
sys.path.insert(0, str(root_dir))

from olake.lakehouse import ObservationLakehouse, ObservationAnalyzer

# initialize lakehouse (assumes that it is already populated)
lakehouse = ObservationLakehouse(
    warehouse_path="../warehouse",
    catalog_db_path="../iceberg_catalog.db"
)

analyzer = ObservationAnalyzer(lakehouse)

iceberg_path = lakehouse.load_observations_table().metadata_location

## Execute a DuckDB query on the lakehouse

In [26]:
analyzer.conn.execute(f"""
    SELECT data_set_id, problem_id FROM iceberg_scan('{iceberg_path}')
    GROUP BY data_set_id, problem_id
""").fetchdf() # return as Pandas DF for display

,data_set_id,problem_id
0,MBPP,mbpp_770_odd_num_sum
1,MBPP,mbpp_593_removezero_ip
2,MBPP,mbpp_259_maximize_elements
3,HumanEval,HumanEval_67_fruit_distribution
4,MBPP,mbpp_392_get_max_sum
...,...,...
504,HumanEval,HumanEval_99_closest_integer
505,MBPP,mbpp_468_max_product
506,MBPP,mbpp_120_max_product_tuple
507,MBPP,mbpp_731_lateralsurface_cone


## Behavioral Clustering (DuckDB -> Pandas DF)

_Note that the clusters are ranked by "cluster size", so the first results acts as a consensus oracle._

In [27]:
problem_id = "mbpp_798__sum" # for given problem

query = f"""-- CTE to create a single, canonical "fingerprint" for each commit.
        WITH v_behavior_signatures AS (
            SELECT
                run_id,
                problem_id,
                test_id,
                implementation_id,
                -- Get JSON string representation
                to_json(
                    list(
                        output ORDER BY step_id
                    )
                ) AS output_sequence_signature
            FROM
                iceberg_scan('{iceberg_path}')
            WHERE
                problem_id = '{problem_id}'
            GROUP BY
                run_id, problem_id, test_id, implementation_id
        ), commit_fingerprints AS (
            SELECT
                run_id,
                problem_id,
                implementation_id,
                -- Create a sorted array of all behavior signatures for this commit.
                array_agg(output_sequence_signature ORDER BY test_id) AS behavior_fingerprint,
                count(test_id) as test_size
            FROM
                v_behavior_signatures
            GROUP BY
                run_id, problem_id, implementation_id
        )
        -- Now, group commits by their identical fingerprints.
        SELECT
            -- cluster of behaviorally equivalent implementations.
            array_agg(implementation_id) AS equivalent_commits_cluster,
            -- The fingerprint that defines this cluster.
            behavior_fingerprint,
            -- How many commits are in this cluster.
            count(*) AS cluster_size,
            MAX(test_size) as test_size
        FROM
            commit_fingerprints
        GROUP BY
            behavior_fingerprint
        ORDER BY
            cluster_size DESC
        """
analyzer.conn.execute(query).fetchdf() # return as Pandas DF for display

,equivalent_commits_cluster,behavior_fingerprint,cluster_size,test_size
0,[7bc3388a-cf85-44af-a36e-df81ad44e824_original...,"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",16,247
1,[22e893ad-06f9-4cfe-8079-4a94dec5cb9f_original...,"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",9,247
2,[7a509465-b5d1-436d-9fef-9f62dcda3c06_original...,"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",2,247
3,[10516539-eaa7-423b-bb0a-e7c6002a0371_original...,"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",2,247
4,[097e859c-5d20-4c4e-8b9f-65649764272f_original_0],"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",1,247
5,[f6f64204-4abf-43e2-bdce-a9c7574e6766_original_0],"[[""$CUT@Problem@0"",""[]"",""$EXCEPTION@java.lang....",1,247
6,[13617523-1a52-4a42-a9d1-b07b0a0b1833_original_0],"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",1,247
7,[2d32ca59-c5c3-473a-b79b-ce719aee93be_original_0],"[[""$CUT@Problem@0"",""[]"",""0""], [""$CUT@Problem@0...",1,247


## SRM Output View (DuckDB -> Pandas DF)

In [28]:
problem_id = "mbpp_798__sum" # for given problem

query = f"""-- CTE to fetch outputs.
    WITH test_sequences AS (
        SELECT
            --run_id,
            --problem_id,
            test_id,
            step_id,
            implementation_id,
            --operation,
            --inputs,
            output
        FROM
            iceberg_scan('{iceberg_path}')
        WHERE
            problem_id = '{problem_id}'
        ORDER BY
            test_id, step_id
    )
    -- Now, pivot the prepared data to create SRM output view
    PIVOT test_sequences
    ON implementation_id
    USING FIRST(output)
    -- GROUP BY: Specifies the column that will serve as the row identifier for the matrix.
    GROUP BY
        test_id, step_id
    ORDER BY
        test_id, step_id
    """
analyzer.conn.execute(query).fetchdf() # return as Pandas DF for display

,test_id,step_id,01e02ed1-29a7-454f-8aac-1e30519d9291_original_0,02041a08-238d-4236-9e1f-7d8ce1e68521_original_0,07719ffb-635a-4d68-9596-81a3396e3bfc_original_0,097e859c-5d20-4c4e-8b9f-65649764272f_original_0,0cac5c5f-0c88-471a-a971-7e1e25253f37_original_0,0f12a5d4-9482-45b1-a1b8-5e2b862555e6_original_0,10516539-eaa7-423b-bb0a-e7c6002a0371_original_0,12bf5d5e-52e3-4458-a205-7238989bde40_original_0,...,a7357a39-0860-47f8-90e6-a9619b4b9ebe_original_0,c12897b6-8175-47e2-b0b2-3c49b0ce26fd_original_0,c4fa75d4-7dc4-4c65-9a62-5bfcd9ead45b_original_0,d733addf-a8ba-4b70-bd44-48be36b83e18_original_0,edee5d61-d128-4746-8d86-81b7c03f1301_original_0,f0437b8f-78ca-4ffc-95df-48bd5dea9060_original_0,f33a6433-454e-4c92-b8a5-7b02a9ec8aac_original_0,f3f6a666-28ed-4f9f-876f-7a9fba939a71_original_0,f6f64204-4abf-43e2-bdce-a9c7574e6766_original_0,f92502ef-7cf0-4d06-bc4a-05ed871acf9d_original_0
0,gemma3latest0_testSumEmptyArray(),0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,...,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0
1,gemma3latest0_testSumEmptyArray(),1,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2,gemma3latest0_testSumEmptyArray(),2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,$EXCEPTION@java.lang.IllegalArgumentException@...,0
3,gemma3latest0_testSumLargeArray(),0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,...,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0
4,gemma3latest0_testSumLargeArray(),1,[null],[null],[null],[null],[null],[null],[null],[null],...,[null],[null],[null],[null],[null],[null],[null],[null],[null],[null]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860,test0(),1,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
861,test1(),0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,...,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0
862,test1(),1,50,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
863,test2(),0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,...,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0,$CUT@Problem@0


## Three-way-join (Observations+Tests+Implementations)

_Note: Test code is contained in column **test_code** and the implementation code is stored in **program_code**._

In [31]:
# load all three tables and their metadata
observations_table = lakehouse.load_observations_table()
code_implementations_table = lakehouse.load_code_table()
tests_table = lakehouse.load_test_table()

 # for given problem
data_set_id = "MBPP"
problem_id = "mbpp_798__sum"

query = f"""-- join all three tables
        SELECT 
            o.*,
            i.source_code as program_code,
            t.source_code as test_code,
            i.language
        FROM iceberg_scan('{observations_table.metadata_location}') o
        INNER JOIN iceberg_scan('{code_implementations_table.metadata_location}') i
            ON o.implementation_id = i.implementation_id
            AND o.data_set_id = i.data_set_id
            AND o.problem_id = i.problem_id
        INNER JOIN iceberg_scan('{tests_table.metadata_location}') t
            ON o.test_id = t.test_id
            AND o.data_set_id = t.data_set_id
            AND o.problem_id = t.problem_id
        WHERE o.data_set_id = '{data_set_id}'
            AND o.problem_id = '{problem_id}'
            AND i.data_set_id = '{data_set_id}'
            AND i.problem_id = '{problem_id}'
            AND t.data_set_id = '{data_set_id}'
            AND t.problem_id = '{problem_id}'
        """
analyzer.conn.execute(query).fetchdf() # return as Pandas DF for display

,data_set_id,problem_id,implementation_id,test_id,implementation_hash,test_hash,run_id,environment_id,step_id,operation,...,memory_used_mb,branch_coverage_percent,created_at,git_commit_hash,ci_pipeline_id,researcher_name,specified_oracle,program_code,test_code,language
0,MBPP,mbpp_798__sum,12bf5d5e-52e3-4458-a205-7238989bde40_original_0,llama31latest8_complexCase_LargeNumbers(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,0ddd6bdc-f4f3-4024-9bd4-d7cbda7ac8a6,2,public static long llama31latest.Main.Sum(java...,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package llama31latest;\n\nimport java.util.*;\...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""java.uti...",java
1,MBPP,mbpp_798__sum,12bf5d5e-52e3-4458-a205-7238989bde40_original_0,gemma3latest3_testSumPositiveNumbers(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,0ddd6bdc-f4f3-4024-9bd4-d7cbda7ac8a6,1,public static java.util.List java.util.Arrays....,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package llama31latest;\n\nimport java.util.*;\...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""Problem""...",java
2,MBPP,mbpp_798__sum,07719ffb-635a-4d68-9596-81a3396e3bfc_original_0,qwen25latest7_testSumHappyCase(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,43ec2055-3510-4716-a4f5-626f29502e46,1,public java.util.ArrayList(java.util.Collection),...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package gemma3latest;\n\nimport java.util.*;\n...,"{""cells"":{""A1"":{},""B1"":""asList"",""C1"":""java.uti...",java
3,MBPP,mbpp_798__sum,edee5d61-d128-4746-8d86-81b7c03f1301_original_0,llama31latest7_sumArrayListEmptyArray(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,11571967-775d-4c9e-8357-12bb840d4b03,0,public java.util.ArrayList(),...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package gemma3latest;\n\nimport java.util.*;\n...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""java.uti...",java
4,MBPP,mbpp_798__sum,0cac5c5f-0c88-471a-a971-7e1e25253f37_original_0,llama31latest0_difficultCase_NegativeNumbers_R...,,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,d0573c04-2e91-43c9-83ed-42aeceb3c36f,3,public boolean java.util.ArrayList.add(java.la...,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package gemma3latest;\n\nimport java.util.*;\n...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""java.uti...",java
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28540,MBPP,mbpp_798__sum,22e893ad-06f9-4cfe-8079-4a94dec5cb9f_original_0,gemma3latest2_testSumMixedPositiveNegativeNumb...,,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,6719f8bd-87ff-4f15-beef-75ae77776389,3,public static long gemma3latest.Problem.Sum(ja...,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package gemma3latest;\n\nimport java.util.*;\n...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""Problem""...",java
28541,MBPP,mbpp_798__sum,f3f6a666-28ed-4f9f-876f-7a9fba939a71_original_0,llama31latest2_happyCaseLargeNumbers(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,bcb9e890-759f-4be2-973b-ce3d1f76f5e2,1,public boolean java.util.ArrayList.add(java.la...,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package gemma3latest;\n\nimport java.util.*;\n...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""java.uti...",java
28542,MBPP,mbpp_798__sum,1dfa8768-b72b-4a5d-9017-04006f113cb1_original_0,qwen25latest4_testSumHappyCase(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,c8652647-db69-4db0-b1b5-039c3aee2ee6,0,public java.util.ArrayList(),...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package llama31latest;\n\nimport java.util.*;\...,"{""cells"":{""A1"":{},""B1"":""create"",""C1"":""java.uti...",java
28543,MBPP,mbpp_798__sum,12bf5d5e-52e3-4458-a205-7238989bde40_original_0,llama31latest8_edgeCase_MixedSigns(),,,14c387a3-dde2-4b9a-b1ee-deae5f8ac0de,0ddd6bdc-f4f3-4024-9bd4-d7cbda7ac8a6,3,public boolean java.util.ArrayList.add(java.la...,...,NaN,NaN,2025-11-18 11:33:56.535447,None,None,None,False,package llama31latest;\n\nimport java.util.*;\.